In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Model, load_model # for assembling a Neural Network model
from keras.layers import Input, Dense, Embedding, Reshape, Concatenate, Flatten, Dropout # for adding layers
from keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, ReLU, LeakyReLU # for adding layers
from tensorflow.keras.utils import plot_model # for plotting model diagram
from tensorflow.keras.optimizers import Adam # for model optimization 
from scipy.stats import entropy
from collections import Counter 
import math

# Data manipulation
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Visualization
import matplotlib 
import matplotlib.pyplot as plt # for data visualizationa
print('matplotlib: %s' % matplotlib.__version__) # print version
import graphviz # for showing model diagram
print('graphviz: %s' % graphviz.__version__) # print version


import pandas as pd
main_dir='framework'

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense

Tensorflow/Keras: 2.8.0
numpy: 1.22.3
matplotlib: 3.6.0
graphviz: 0.20.1


In [2]:
def latent_vector(latent_dim, n, n_cats=36, m_cats=3):
    
    # Generate points in the latent space
    latent_input = np.random.randn(latent_dim * n)
    
    # Reshape into a batch of inputs for the network
    latent_input = latent_input.reshape(n, latent_dim)
    
    # Generate category labels 
    cat_labels = np.random.randint(0, n_cats, n)
    cat_labels2 = np.random.randint(0, m_cats, n)
    
    return [latent_input, cat_labels, cat_labels2]


In [3]:
testcount=360

In [97]:
# Generate latent points
latent_points, _, _ = latent_vector(100, testcount)#100dim100個

# Specify labels that we want (0-9 repeated 10 times)
# labels = np.asarray([x for _ in range(10) for x in range(0,36)])#100個擠到幾
po=[]
for y in range(36):
    for x in range(10):
        po.append(y)
labels = np.asarray(po)
# la0 = np.asarray([x for _ in range(20) for x in range(0,1)])#100個擠到幾
la1 = np.asarray([x for _ in range(testcount) for x in range(2,3)])#100個擠到幾
# la2 = np.asarray([x for _ in range(20) for x in range(2,3)])#100個擠到幾
# labels2 = np.concatenate((la0, la1, la2), axis = 0)
# Load previously saved generator model


model = load_model(main_dir+'/0307-2norelulstm_cgan_lenclus_50epoch-hotLN.h5')

# Generate images
gen_imgs  = model.predict([latent_points, labels, la1])

# Scale from [-1, 1] to [0, 1]
# gen_imgs = (gen_imgs + 1) / 2.0

# # Display images
# fig, axs = plt.subplots(10, 38, sharey=False, tight_layout=True, figsize=(16,16), facecolor='white')
# k=0
# for i in range(0,10):
#     for j in range(0,38):
#         axs[i,j].matshow(gen_imgs[k], cmap='gray')
#         axs[0,j].set(title=labels[k])
#         axs[i,j].axis('off')
#         k=k+1
# plt.show() 

In [98]:
gen_imgs.shape
genin = gen_imgs.reshape(testcount,38,71)
genin.shape

(360, 38, 71)

In [99]:
from keras.models import load_model
decoder = load_model('0307noembedding0-54000_25-71dc-hotLN.h5')

In [100]:
deout = decoder(genin)

In [101]:
p=[]
pp=[]
for i in range(testcount):
    p=[]
    for j in range(38):
        p.append(np.argmax(deout[i][j]))
    pp.append(p)
pp

[[3256,
  3264,
  2166,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1342,
  0,
  0,
  0],
 [3009,
  2164,
  779,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [2326,
  3441,
  867,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  162,
  0,
  0,
  0],
 [505,
  581,
  2750,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  734,
  0,
  0,
  0],
 [1027,
  2755,
  1554,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1342,
  0,
  0,


# 算方法一旅程中所有點集合的長度

In [102]:
#長度誤差RMSE
def longRMSE(targetshape,allshape):
    calcu=0
    for i in range(len(allshape)):
        se=(allshape[i]-targetshape)*(allshape[i]-targetshape)
        calcu+=se
    return math.sqrt(calcu/len(allshape))

In [103]:
def tocate(num):
    o=[]
    df=pd.read_excel('poi-TKY.xlsx')
    for i in range(len(num)):
        for j in range(len(df)):
            if num[i]==df.poiID[j]:
                o.append(df.cate[j])
                break
    return o

In [104]:
def failcount(L):
    fcount=0
    for i in range(len(L)):
        for j in set(L[i]):
            if L[i].count(j)>1:
                fcount+=1
                break
    return fcount

In [105]:
p1=[]
pppp1=[]
for i in range(len(pp)):
    p=[]
    for j in range(38):
        if pp[i][j]!=0:
            p.append(pp[i][j])
            pppp1.append(pp[i][j])
    p1.append(p)

In [106]:
plen1=[]
for i in range(testcount):
    plen1.append(len(p1[i]))

In [107]:
plen1=np.array(plen1)
plen1_36_10=plen1.reshape(36,10)
plen1_36_10

array([[ 4,  3,  4,  4,  4,  6,  4,  4,  4,  4],
       [ 6, 10, 10,  7,  6,  8,  9,  5,  7,  9],
       [ 6, 11, 12,  9, 10,  6,  6, 10, 10, 11],
       [11, 10, 11, 11, 10, 10, 15, 11, 13, 10],
       [ 9, 11,  8, 17, 10, 10, 13,  9, 10, 13],
       [18, 19, 15, 16, 11, 17, 18, 13, 13, 16],
       [15, 10, 13, 14,  9,  9, 10, 12, 13, 17],
       [11,  9, 15, 19, 11, 19, 14, 15, 13, 13],
       [11, 19, 17, 17, 16, 14, 16, 20, 12, 17],
       [20, 19, 18, 18, 17, 20, 22, 19, 19, 17],
       [18, 16, 16, 20, 17, 20, 19, 18, 19, 18],
       [17, 21, 19, 23, 19, 20, 20, 18, 22, 20],
       [23, 22, 16, 24, 26, 20, 22, 23, 19, 20],
       [21, 21, 25, 20, 24, 21, 23, 21, 21, 21],
       [25, 23, 23, 24, 25, 23, 24, 21, 24, 26],
       [26, 24, 21, 27, 22, 23, 22, 24, 24, 25],
       [27, 25, 24, 29, 28, 27, 25, 27, 27, 28],
       [28, 24, 25, 24, 22, 25, 25, 25, 23, 29],
       [29, 30, 29, 26, 25, 25, 31, 24, 25, 31],
       [27, 26, 28, 29, 28, 29, 29, 30, 28, 29],
       [28, 27, 26, 

In [108]:
sum(plen1)

8753

In [109]:
len(pppp1)

8753

In [110]:
pppp1uni=np.unique(pppp1)
pppp1unicate=tocate(pppp1uni)

In [111]:
len(pppp1uni)

1941

In [112]:
count1=[]
l1=Counter(sorted(pppp1))
values1=l1.values()
for i in values1:
    count1.append(i)

In [113]:
pppp1cate=tocate(pppp1)
pppp1cate

[2,
 2,
 5,
 2,
 4,
 5,
 4,
 2,
 4,
 2,
 5,
 4,
 2,
 2,
 5,
 0,
 3,
 2,
 2,
 2,
 2,
 6,
 12,
 12,
 5,
 3,
 5,
 6,
 5,
 2,
 3,
 3,
 2,
 4,
 4,
 2,
 5,
 3,
 2,
 2,
 5,
 3,
 4,
 2,
 3,
 2,
 5,
 3,
 4,
 2,
 3,
 6,
 5,
 12,
 2,
 12,
 5,
 2,
 6,
 2,
 2,
 2,
 12,
 5,
 12,
 5,
 12,
 3,
 6,
 4,
 5,
 2,
 5,
 12,
 2,
 3,
 2,
 5,
 6,
 5,
 5,
 2,
 4,
 2,
 6,
 12,
 12,
 12,
 5,
 2,
 3,
 2,
 3,
 12,
 12,
 5,
 12,
 2,
 3,
 2,
 4,
 5,
 6,
 3,
 4,
 4,
 2,
 3,
 5,
 2,
 4,
 4,
 5,
 2,
 6,
 12,
 12,
 5,
 2,
 3,
 2,
 6,
 5,
 5,
 2,
 3,
 2,
 4,
 2,
 12,
 12,
 12,
 5,
 12,
 5,
 2,
 3,
 3,
 4,
 2,
 2,
 12,
 2,
 12,
 12,
 12,
 5,
 2,
 4,
 6,
 2,
 3,
 3,
 12,
 12,
 5,
 2,
 5,
 2,
 6,
 2,
 2,
 12,
 12,
 12,
 12,
 3,
 2,
 6,
 2,
 3,
 5,
 2,
 2,
 2,
 3,
 3,
 12,
 2,
 2,
 2,
 5,
 3,
 3,
 12,
 12,
 12,
 3,
 6,
 2,
 2,
 2,
 6,
 2,
 5,
 5,
 12,
 12,
 7,
 2,
 5,
 2,
 7,
 3,
 2,
 5,
 12,
 5,
 12,
 4,
 3,
 2,
 6,
 6,
 5,
 3,
 2,
 2,
 12,
 5,
 2,
 2,
 5,
 2,
 3,
 6,
 2,
 2,
 2,
 5,
 4,
 2,
 2,
 6,
 2,
 2,
 3,
 5,
 5,
 12,


In [114]:
cate1num = Counter(sorted(pppp1cate))
cate1numvalues = cate1num.values()
for i in cate1numvalues:
    print(i)
cate1num

40
257
3260
1277
949
1058
1005
35
209
44
2
7
607
3


Counter({0: 40,
         1: 257,
         2: 3260,
         3: 1277,
         4: 949,
         5: 1058,
         6: 1005,
         7: 35,
         8: 209,
         9: 44,
         10: 2,
         11: 7,
         12: 607,
         13: 3})

In [115]:
p1forentro=[]
for i in range(len(pppp1uni)):
    p1forentro.append(count1[i]/cate1num[pppp1unicate[i]])

In [116]:
base = math.e  # work in units of bits
# pk = np.array([1/2, 1/2])  # fair coin
H1 = entropy(p1forentro)
H1

6.02957977233644

In [117]:
failcount(p1)

188

In [118]:
alllongerror1=[]
for i in range(36):
    space1=[]
    space1.append(i+3)
    space1.append(longRMSE(i+3,plen1_36_10[i]))
    print(longRMSE(i+3,plen1_36_10[i]))
    alllongerror1.append(space1)
alllongerror1

1.3038404810405297
4.06201920231798
4.636809247747852
5.422176684690384
4.732863826479693
7.987490219086343
4.09878030638384
4.9899899799498595
5.594640292279746
7.049822692805827
5.282045058497703
6.1400325732035
7.035623639735144
6.0
6.928203230275509
6.06630035524124
7.842193570679061
5.385164807134504
7.00713921654194
6.395310782127793
5.0990195135927845
3.96232255123179
4.301162633521313
4.219004621945797
4.09878030638384
4.135214625627067
3.1144823004794873
2.1447610589527217
2.6832815729997477
1.8973665961010275
2.345207879911715
1.1832159566199232
1.140175425099138
1.224744871391589
0.9486832980505138
1.6431676725154984


[[3, 1.3038404810405297],
 [4, 4.06201920231798],
 [5, 4.636809247747852],
 [6, 5.422176684690384],
 [7, 4.732863826479693],
 [8, 7.987490219086343],
 [9, 4.09878030638384],
 [10, 4.9899899799498595],
 [11, 5.594640292279746],
 [12, 7.049822692805827],
 [13, 5.282045058497703],
 [14, 6.1400325732035],
 [15, 7.035623639735144],
 [16, 6.0],
 [17, 6.928203230275509],
 [18, 6.06630035524124],
 [19, 7.842193570679061],
 [20, 5.385164807134504],
 [21, 7.00713921654194],
 [22, 6.395310782127793],
 [23, 5.0990195135927845],
 [24, 3.96232255123179],
 [25, 4.301162633521313],
 [26, 4.219004621945797],
 [27, 4.09878030638384],
 [28, 4.135214625627067],
 [29, 3.1144823004794873],
 [30, 2.1447610589527217],
 [31, 2.6832815729997477],
 [32, 1.8973665961010275],
 [33, 2.345207879911715],
 [34, 1.1832159566199232],
 [35, 1.140175425099138],
 [36, 1.224744871391589],
 [37, 0.9486832980505138],
 [38, 1.6431676725154984]]

In [119]:
plen=[]
for i in range(testcount):
    ltemp=0
    for j in range(38):
        if pp[i][j]!=0:
            ltemp+=1
        else:
            break
    plen.append(ltemp)

In [120]:
plen

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 6,
 3,
 5,
 5,
 7,
 5,
 4,
 6,
 7,
 5,
 7,
 11,
 6,
 6,
 3,
 4,
 6,
 6,
 8,
 10,
 9,
 8,
 10,
 9,
 9,
 7,
 9,
 8,
 9,
 8,
 10,
 7,
 13,
 6,
 6,
 10,
 7,
 7,
 11,
 15,
 15,
 13,
 12,
 8,
 14,
 15,
 7,
 11,
 14,
 12,
 7,
 6,
 10,
 7,
 6,
 8,
 7,
 9,
 8,
 8,
 7,
 13,
 11,
 7,
 15,
 12,
 11,
 7,
 11,
 2,
 14,
 15,
 7,
 14,
 6,
 11,
 16,
 8,
 11,
 14,
 17,
 16,
 14,
 15,
 18,
 19,
 8,
 18,
 16,
 15,
 14,
 14,
 17,
 16,
 15,
 8,
 17,
 17,
 11,
 14,
 17,
 17,
 15,
 16,
 16,
 19,
 17,
 19,
 18,
 18,
 15,
 15,
 19,
 20,
 16,
 14,
 20,
 16,
 17,
 19,
 19,
 21,
 20,
 20,
 18,
 17,
 19,
 14,
 19,
 21,
 20,
 14,
 21,
 19,
 22,
 21,
 20,
 20,
 18,
 21,
 20,
 13,
 19,
 20,
 19,
 19,
 19,
 20,
 22,
 21,
 21,
 22,
 22,
 23,
 6,
 22,
 25,
 22,
 23,
 19,
 22,
 21,
 20,
 21,
 22,
 21,
 23,
 23,
 21,
 22,
 24,
 23,
 23,
 19,
 23,
 22,
 22,
 20,
 22,
 24,
 23,
 23,
 25,
 25,
 24,
 22,
 24,
 22,
 24,
 23,
 23,
 25,
 24,
 27,
 25,
 26,
 23,
 24,
 26,
 26,
 27,
 26,

In [121]:
pp=np.array(pp)
pp.shape
ppp=pp.reshape(testcount*38)

In [122]:
from collections import Counter 
print(Counter(sorted(ppp)))

Counter({0: 4927, 192: 343, 959: 139, 3645: 66, 617: 60, 3372: 47, 272: 42, 20: 40, 3427: 38, 2567: 37, 746: 35, 1191: 35, 2977: 33, 2033: 31, 386: 30, 684: 30, 816: 30, 1070: 30, 2589: 30, 3317: 29, 170: 28, 593: 27, 3501: 27, 113: 26, 701: 26, 782: 26, 2199: 26, 241: 25, 35: 24, 1318: 24, 2357: 24, 292: 23, 3133: 23, 375: 22, 800: 22, 1152: 22, 509: 21, 818: 21, 1220: 21, 505: 20, 524: 20, 802: 20, 2204: 20, 425: 19, 526: 19, 1214: 19, 2614: 19, 3131: 19, 3264: 19, 243: 18, 307: 18, 813: 18, 883: 18, 1774: 18, 1800: 18, 1964: 18, 2135: 18, 3216: 18, 3368: 18, 124: 17, 155: 17, 264: 17, 287: 17, 534: 17, 777: 17, 3082: 17, 3398: 17, 434: 16, 647: 16, 703: 16, 824: 16, 936: 16, 1062: 16, 3132: 16, 3295: 16, 289: 15, 577: 15, 582: 15, 942: 15, 945: 15, 2687: 15, 87: 14, 284: 14, 591: 14, 765: 14, 939: 14, 1048: 14, 1263: 14, 1270: 14, 2885: 14, 11: 13, 138: 13, 266: 13, 351: 13, 442: 13, 710: 13, 1098: 13, 1194: 13, 1339: 13, 1459: 13, 1852: 13, 2326: 13, 2429: 13, 2834: 13, 2873: 13, 3

In [123]:
c=[]

for i in range(testcount):
    c.append(tocate(pp[i]))

In [124]:
c=np.array(c)
c=c.reshape(testcount*38)

In [125]:
Counter(sorted(c))

Counter({0: 40,
         1: 257,
         2: 3260,
         3: 1277,
         4: 949,
         5: 1058,
         6: 1005,
         7: 35,
         8: 209,
         9: 44,
         10: 2,
         11: 7,
         12: 607,
         13: 3,
         14: 4927})

In [126]:
pppp=[]
pppp360_plen=[]
for i in range(len(pp)):
    tt=[]
    for j in range(plen[i]):
        tt.append(pp[i][j])
        pppp.append(pp[i][j])
#     print(len(tt))
    pppp360_plen.append(tt)
pppp360_plen

[[3256, 3264, 2166],
 [3009, 2164, 779],
 [2326, 3441, 867],
 [505, 581, 2750],
 [1027, 2755, 1554],
 [541, 1823, 2722],
 [1865, 2380, 840],
 [816, 2361, 3396],
 [252, 1593, 2082],
 [3582, 3402, 591],
 [3553, 2698, 601, 285, 3398],
 [1451, 534, 3366, 2885, 970, 3317],
 [1, 797, 684],
 [2963, 1390, 685, 2380, 11],
 [929, 170, 2567, 3320, 8],
 [2875, 614, 2834, 860, 609, 2199, 617],
 [806, 234, 2206, 1318, 2968],
 [816, 2729, 284, 1424],
 [3577, 72, 3065, 2766, 777, 2999],
 [1526, 505, 2466, 734, 113, 46, 2027],
 [607, 3010, 3030, 2020, 1555],
 [1206, 2401, 1210, 505, 1112, 2199, 617],
 [2326, 1383, 3139, 1701, 942, 2145, 2199, 536, 617, 617, 617],
 [3270, 940, 225, 1318, 230, 2494],
 [1112, 2380, 3114, 2207, 114, 287],
 [1547, 297, 2865],
 [314, 375, 2933, 3060],
 [612, 2744, 1427, 813, 266, 3431],
 [524, 73, 2151, 684, 2246, 375],
 [159, 701, 3624, 2024, 1614, 3169, 2567, 553],
 [2283, 281, 877, 791, 2121, 3373, 1752, 942, 1259, 631],
 [1194, 1062, 2221, 1942, 3096, 2252, 1266, 1220, 7

In [127]:
ppppuni=np.unique(pppp)
ppppunicate=tocate(ppppuni)

In [128]:
len(ppppuni)

1894

In [129]:
len(ppppunicate)

1894

In [130]:
ppppuni

array([   1,    2,    3, ..., 3672, 3679, 3680], dtype=int64)

In [131]:
ppppunicate

[2,
 2,
 3,
 6,
 2,
 4,
 6,
 6,
 5,
 4,
 2,
 2,
 6,
 2,
 8,
 2,
 2,
 2,
 12,
 3,
 6,
 3,
 2,
 2,
 3,
 4,
 5,
 5,
 6,
 2,
 5,
 2,
 2,
 7,
 2,
 2,
 4,
 8,
 2,
 5,
 1,
 5,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 4,
 2,
 12,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 6,
 2,
 2,
 2,
 2,
 3,
 2,
 4,
 0,
 2,
 0,
 6,
 3,
 2,
 8,
 2,
 6,
 2,
 9,
 3,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 6,
 4,
 6,
 7,
 6,
 12,
 5,
 3,
 2,
 3,
 3,
 2,
 2,
 6,
 2,
 2,
 1,
 3,
 3,
 6,
 2,
 13,
 12,
 7,
 2,
 3,
 1,
 3,
 6,
 6,
 2,
 6,
 3,
 6,
 6,
 3,
 3,
 3,
 2,
 2,
 3,
 6,
 3,
 2,
 3,
 2,
 4,
 3,
 4,
 4,
 4,
 2,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 2,
 2,
 3,
 4,
 6,
 3,
 4,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 5,
 2,
 3,
 5,
 2,
 7,
 3,
 2,
 3,
 3,
 3,
 2,
 5,
 2,
 4,
 6,
 4,
 2,
 2,
 6,
 5,
 2,
 2,
 5,
 2,
 6,
 5,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 6,
 3,
 5,
 2,
 2,
 5,
 4,
 6,
 2,
 4,
 6,
 2,
 2,
 2,
 4,
 3,
 6,
 2,
 2,
 2,
 2,
 2,
 1,
 6,
 8,
 3,
 2,
 6,
 2,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 4,
 6,
 4,
 3,
 6,

In [132]:
# ppppunicate=np.array(ppppunicate)
# result = np.where(ppppunicate == 13)
# print(result)

In [133]:
len(pppp)

7335

In [134]:
print(Counter(sorted(pppp)))

Counter({20: 40, 272: 39, 2567: 34, 746: 33, 816: 30, 386: 28, 684: 27, 593: 26, 782: 26, 113: 24, 241: 24, 701: 24, 1318: 24, 35: 23, 617: 23, 2357: 23, 3133: 23, 292: 22, 375: 22, 1191: 22, 3317: 22, 800: 21, 818: 21, 505: 20, 509: 20, 526: 19, 802: 19, 1152: 19, 1220: 19, 2199: 19, 2614: 19, 243: 18, 425: 18, 883: 18, 1214: 18, 1774: 18, 1964: 18, 3264: 18, 287: 17, 534: 17, 813: 17, 1800: 17, 3082: 17, 3131: 17, 3368: 17, 3398: 17, 155: 16, 647: 16, 777: 16, 264: 15, 307: 15, 434: 15, 524: 15, 582: 15, 936: 15, 1062: 15, 1070: 15, 124: 14, 192: 14, 577: 14, 591: 14, 703: 14, 1048: 14, 2687: 14, 2885: 14, 3216: 14, 11: 13, 284: 13, 289: 13, 442: 13, 710: 13, 765: 13, 824: 13, 939: 13, 942: 13, 1270: 13, 1339: 13, 1852: 13, 2326: 13, 2429: 13, 2834: 13, 3295: 13, 3381: 13, 3427: 13, 100: 12, 138: 12, 351: 12, 742: 12, 1263: 12, 2873: 12, 2933: 12, 3132: 12, 3396: 12, 81: 11, 170: 11, 177: 11, 260: 11, 267: 11, 380: 11, 465: 11, 542: 11, 776: 11, 1194: 11, 1459: 11, 1665: 11, 1723: 11

In [135]:
count=[]
l=Counter(sorted(pppp))
values=l.values()
for i in values:
    count.append(i)

In [136]:
ppppcate=tocate(pppp)
ppppcate

[2,
 2,
 5,
 4,
 5,
 4,
 2,
 4,
 2,
 4,
 2,
 2,
 0,
 3,
 2,
 2,
 2,
 6,
 3,
 5,
 6,
 2,
 3,
 3,
 4,
 4,
 2,
 3,
 2,
 2,
 3,
 4,
 2,
 3,
 2,
 3,
 4,
 2,
 3,
 6,
 5,
 2,
 6,
 2,
 3,
 6,
 4,
 5,
 2,
 2,
 3,
 2,
 5,
 6,
 5,
 2,
 4,
 2,
 6,
 12,
 12,
 5,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 4,
 6,
 3,
 4,
 4,
 2,
 3,
 2,
 4,
 4,
 5,
 2,
 6,
 12,
 2,
 3,
 2,
 6,
 5,
 2,
 3,
 2,
 4,
 2,
 12,
 12,
 2,
 3,
 3,
 4,
 2,
 2,
 12,
 2,
 12,
 12,
 12,
 2,
 4,
 6,
 2,
 3,
 3,
 2,
 5,
 2,
 6,
 2,
 2,
 3,
 2,
 6,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 5,
 3,
 3,
 6,
 2,
 2,
 2,
 6,
 2,
 7,
 2,
 5,
 2,
 7,
 3,
 2,
 5,
 4,
 3,
 2,
 6,
 6,
 5,
 3,
 2,
 2,
 12,
 2,
 2,
 5,
 2,
 3,
 6,
 2,
 2,
 2,
 4,
 2,
 2,
 6,
 2,
 2,
 3,
 5,
 2,
 2,
 2,
 3,
 5,
 2,
 5,
 4,
 5,
 5,
 2,
 2,
 3,
 2,
 2,
 2,
 5,
 6,
 5,
 5,
 2,
 6,
 2,
 6,
 6,
 3,
 2,
 2,
 1,
 6,
 2,
 6,
 3,
 3,
 2,
 2,
 2,
 5,
 2,
 3,
 3,
 1,
 8,
 2,
 4,
 2,
 2,
 6,
 6,
 3,
 4,
 2,
 2,
 4,
 3,
 2,
 6,
 2,
 3,
 12,
 5,
 2,
 2,
 6,
 2,
 3,
 2,
 12,
 12,
 6,
 2,
 3,
 8,
 2,
 2,

In [137]:
catenum = Counter(sorted(ppppcate))
catenumvalues = catenum.values()
for i in catenumvalues:
    print(i)
catenum

38
246
3052
1145
902
615
943
35
195
41
2
6
112
3


Counter({0: 38,
         1: 246,
         2: 3052,
         3: 1145,
         4: 902,
         5: 615,
         6: 943,
         7: 35,
         8: 195,
         9: 41,
         10: 2,
         11: 6,
         12: 112,
         13: 3})

In [138]:
pforentro=[]
for i in range(len(ppppuni)):
    pforentro.append(count[i]/catenum[ppppunicate[i]])

In [139]:
base = math.e  # work in units of bits
# pk = np.array([1/2, 1/2])  # fair coin
H = entropy(pforentro)
H

6.109982077801774

In [140]:
failcount(pppp360_plen)

76

In [141]:
plen=np.array(plen)
plen36_10=plen.reshape(36,10)
plen36_10

array([[ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3],
       [ 5,  6,  3,  5,  5,  7,  5,  4,  6,  7],
       [ 5,  7, 11,  6,  6,  3,  4,  6,  6,  8],
       [10,  9,  8, 10,  9,  9,  7,  9,  8,  9],
       [ 8, 10,  7, 13,  6,  6, 10,  7,  7, 11],
       [15, 15, 13, 12,  8, 14, 15,  7, 11, 14],
       [12,  7,  6, 10,  7,  6,  8,  7,  9,  8],
       [ 8,  7, 13, 11,  7, 15, 12, 11,  7, 11],
       [ 2, 14, 15,  7, 14,  6, 11, 16,  8, 11],
       [14, 17, 16, 14, 15, 18, 19,  8, 18, 16],
       [15, 14, 14, 17, 16, 15,  8, 17, 17, 11],
       [14, 17, 17, 15, 16, 16, 19, 17, 19, 18],
       [18, 15, 15, 19, 20, 16, 14, 20, 16, 17],
       [19, 19, 21, 20, 20, 18, 17, 19, 14, 19],
       [21, 20, 14, 21, 19, 22, 21, 20, 20, 18],
       [21, 20, 13, 19, 20, 19, 19, 19, 20, 22],
       [21, 21, 22, 22, 23,  6, 22, 25, 22, 23],
       [19, 22, 21, 20, 21, 22, 21, 23, 23, 21],
       [22, 24, 23, 23, 19, 23, 22, 22, 20, 22],
       [24, 23, 23, 25, 25, 24, 22, 24, 22, 24],
       [23, 23, 25, 

In [142]:
alllongerror=[]
for i in range(36):
    space=[]
    space.append(i+3)
    space.append(longRMSE(i+3,plen36_10[i]))
    print(longRMSE(i+3,plen36_10[i]))
    alllongerror.append(space)
alllongerror

0.0
1.760681686165901
2.4083189157584592
2.932575659723036
2.701851217221259
5.196152422706632
2.04939015319192
2.6832815729997477
4.358898943540674
4.5934736311423405
3.0983866769659336
3.1937438845342623
2.8635642126552705
3.1937438845342623
3.3763886032268267
2.569046515733026
5.300943312279429
1.760681686165901
1.7320508075688772
1.8973665961010275
2.0976176963403033
3.794733192202055
5.549774770204643
10.0
1.7320508075688772
3.687817782917155
5.06951674225463
1.0
10.029955134495866
9.539392014169456
10.232301793829187
11.436782764396638
1.4832396974191326
8.28854631404084
10.700467279516348
7.0285133563222315


[[3, 0.0],
 [4, 1.760681686165901],
 [5, 2.4083189157584592],
 [6, 2.932575659723036],
 [7, 2.701851217221259],
 [8, 5.196152422706632],
 [9, 2.04939015319192],
 [10, 2.6832815729997477],
 [11, 4.358898943540674],
 [12, 4.5934736311423405],
 [13, 3.0983866769659336],
 [14, 3.1937438845342623],
 [15, 2.8635642126552705],
 [16, 3.1937438845342623],
 [17, 3.3763886032268267],
 [18, 2.569046515733026],
 [19, 5.300943312279429],
 [20, 1.760681686165901],
 [21, 1.7320508075688772],
 [22, 1.8973665961010275],
 [23, 2.0976176963403033],
 [24, 3.794733192202055],
 [25, 5.549774770204643],
 [26, 10.0],
 [27, 1.7320508075688772],
 [28, 3.687817782917155],
 [29, 5.06951674225463],
 [30, 1.0],
 [31, 10.029955134495866],
 [32, 9.539392014169456],
 [33, 10.232301793829187],
 [34, 11.436782764396638],
 [35, 1.4832396974191326],
 [36, 8.28854631404084],
 [37, 10.700467279516348],
 [38, 7.0285133563222315]]

# 長度誤差

In [50]:
#長度誤差
def longerror(targetshape,allshape):
    calcu=0
    for i in range(len(allshape)):
        calcu+=abs(allshape[i]-targetshape)
    return calcu/len(allshape)

In [194]:
plen=np.array(plen)

In [195]:
plen36_10=plen.reshape(36,10)

In [196]:
plen36_10

array([[ 2,  4,  3,  3,  3,  3,  4,  4,  3,  3],
       [ 4,  4,  4,  5,  4,  3,  4,  1,  4,  4],
       [ 5,  5,  5,  4,  5,  5,  5,  5,  5,  6],
       [ 8,  9,  7,  8,  9, 10,  8,  6, 10,  8],
       [11,  8,  5,  8, 11, 11, 11, 12,  7,  8],
       [ 8,  9, 12, 11, 11, 10,  9,  9, 11, 10],
       [12, 12, 12, 11, 12, 13, 11, 11, 12, 12],
       [ 3, 12, 12, 12,  7, 11, 11, 11, 13, 11],
       [12, 11, 11, 11,  3, 12, 11, 11, 13, 13],
       [13, 14, 14, 13, 14, 14, 13, 13, 13, 14],
       [15, 14,  4, 15, 15, 14, 14, 15, 14, 15],
       [15, 15, 15, 15, 15, 14, 16, 15, 15, 15],
       [15, 15, 15, 15, 15, 15, 14, 15, 14, 15],
       [ 3, 15, 17, 15, 16, 17, 17, 17, 17, 17],
       [17, 19, 18, 17, 18, 17, 17, 18, 17, 18],
       [17, 18, 17, 19, 18, 12, 18, 15, 16, 17],
       [16, 17, 16, 16, 16, 15, 16, 16, 16, 13],
       [19, 23, 22, 19, 21, 21, 21, 16, 22, 22],
       [22, 18, 21, 20, 21, 20, 22, 20,  7,  2],
       [22, 21, 22, 21, 22, 22, 22, 23, 22, 24],
       [20, 23, 22, 

In [197]:
alllongerror=[]
for i in range(36):
    space=[]
    space.append(i+3)
    space.append(longerror(i+3,plen36_10[i]))
    alllongerror.append(space)
alllongerror

[[3, 0.4],
 [4, 0.5],
 [5, 0.2],
 [6, 2.3],
 [7, 2.6],
 [8, 2.0],
 [9, 2.8],
 [10, 2.3],
 [11, 1.4],
 [12, 1.5],
 [13, 2.3],
 [14, 1.0],
 [15, 0.2],
 [16, 2.1],
 [17, 0.6],
 [18, 1.5],
 [19, 3.3],
 [20, 1.8],
 [21, 4.1],
 [22, 0.5],
 [23, 1.4],
 [24, 3.1],
 [25, 3.1],
 [26, 0.8],
 [27, 1.7],
 [28, 6.3],
 [29, 1.2],
 [30, 1.3],
 [31, 3.1],
 [32, 6.4],
 [33, 2.5],
 [34, 7.9],
 [35, 5.6],
 [36, 5.5],
 [37, 4.6],
 [38, 3.5]]